In [15]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from nltk import word_tokenize
from collections import Counter
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.metrics import accuracy_score

import pandas as pd

import numpy as np

ua = UserAgent(verify_ssl=False)
session = requests.session()

In [7]:
news = []
page_number = 0
for i in tqdm(range(4)):
    page_number += 1
    url = f'https://www.mk.ru/incident/{page_number}/'
    req = session.get(url, headers={'User-Agent': ua.random})
    print(req)
    req.encoding = 'utf-8'
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    each_news = soup.find_all('div', {'class': 'listing-item__content'})
    if each_news:
        print(len(each_news))
    for one_piece in each_news:
        link = one_piece.find('a', {'class': 'listing-preview__content'}).attrs['href']
        news.append(link)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

<Response [200]>
30

 25%|█████████████████████                                                               | 1/4 [00:02<00:06,  2.02s/it]


<Response [200]>


 50%|██████████████████████████████████████████                                          | 2/4 [00:03<00:03,  1.79s/it]

...

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:04<00:01,  1.66s/it]

<Response [200]>
30


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.57s/it]

<Response [200]>
30


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.50s/it]


In [8]:
print(len(news))

120


In [9]:
link = news[10]
req = session.get(link, headers={'User-Agent': ua.random})
req.encoding = 'utf-8'
req = req.text
req = BeautifulSoup(req, 'html.parser')
new_text = req.find('div', {'class': "article__body"}) #.text.lower()
#print(new_text)
new_text = new_text.find_all('p') #.text.lower()
print(new_text)

[<p>Мать пострадавших по профессии повар, отец - дворник, оба приехали из Киргизии. «Двушку» на Зеленом проспекте они сняли год назад. Когда началась пандемия, мужчина потерял большую часть заработка, поэтому платить аренду стало тяжело. На семейном совете было решено сдать одну комнату какому-нибудь земляку.  </p>, <p>В начале сентября нашелся приезжий из Ошской области. Мужчина заявил, что он аниматор и что к нему скоро приедет жена с ребенком. Позже выяснилось, что он придумал эту историю - на самом деле он одинок.</p>, <p>Об ужасной ситуации 29-летняя мать девочек 6 и 7 лет от роду узнала совершенно случайно - 19 октября во время заучивания с детьми стихов. Старшая девочка, увидев соседа по квартире, вышедшего из комнаты в туалет, внезапно замкнулась и стала в волнении грызть ногти, заявив, что боится этого дядю. Женщина поинтересовалась происходящим, но девочка не ответила. </p>, <p>А вторая дочка, более открытая по характеру, как на духу выложила неприятные подробности. Она расск

In [10]:
from tqdm import tqdm

In [11]:
def get_full_news(link_list):
    texts = []
    titles = []
    for link in tqdm(link_list):
        news = []
        req = session.get(link, headers={'User-Agent': ua.random})
        req.encoding = 'utf-8'
        req = req.text
        req = BeautifulSoup(req, 'html.parser')
        title = req.find('h1', {'class': "article__title"})
        titles.append(title)
        new_text = req.find('div', {'class': "article__body"})
        new_text = new_text.find_all('p')
        for p in new_text:
            p = p.text
            news.append(p)
        texts.append(news)
    return titles, texts

In [12]:
titles, texts = get_full_news(news)
if len(texts)==len(titles):
    print('all clear!')

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [02:59<00:00,  1.50s/it]

all clear!


In [13]:
texts[:1]

[['Напомним, 23-го сентября вечером дочь артиста София Конкина пришла в фитнес клуб в Пресненском районе, что неподалёку от съёмной квартиры, в которой последние несколько месяцев она жила с дочерью и своим кавалером, водителем по профессии Михаилом. Девушка сразу направилась в бассейн, где около полуночи загадочным образом утонула. В бессознательном состоянии Софию вытащили из воды и «скорая» доставила её в больницу, но спасти девушку врачам не удалось.',
  'Смерть дочери знаменитого актера получила широкий резонанс из-за массы несостыковок и противоречивых данных об обстоятельствах, предшествующих трагедии. Например, у родственников погибшей возникли подозрения по поводу личности её сожителя, а сам факт гибели здоровой и хорошо плавающей девушки и по сей день выглядит странным. Близкие Софии до сих пор отказываются верить в несчастный случай, даже несмотря на то, что в её организме был обнаружен алкоголь и следы седативного препарата, и настаивают на полноценном расследовании (в том 

In [18]:
df = pd.DataFrame(list(zip(titles, texts)), 
               columns =['titles', 'texts']) 
df

,titles,texts
0,[Раскрыта тайна препарата в организме утонувше...,"[Напомним, 23-го сентября вечером дочь артиста..."
1,"[Девочка сбежала от матери, делавшей из нее ра...","[Наверное, у каждого были прадедушки и прабабу..."
2,[Медбрат украл аппарат УЗИ у больных коронавир...,"[Как стало известно «МК», пропажу аппарата УЗИ..."
3,[Составлен рейтинг мошенничеств с коронавирусом],[Лже-медики. Облапошивание начинается со звонк...
4,[Приговор обрадовал депутата-коммуниста Шереме...,[Ранее в ходе прений прокурор потребовал для Ш...
...,...,...
115,[Подросток умер во время вечеринки на юго-запа...,"[Как стало известно «МК», инцидент произошел в..."
116,[Российский полицейский убил любовницу-трансге...,"[По информации издания, поначалу мужчина пытал..."
117,[Студент МГИМО покончил с собой из-за сессии],"[По информации агентства, его тело обнаружили ..."
118,[Рабочего-расчленителя задержали в Петербурге],"[По версии следственных органов, 5 сентября ме..."


In [19]:
df.to_csv('textbase.csv')

In [ ]:
toy_df = pd.DataFrame({
   'EmployeeId': ['001', '002', '003', '004'],
   'City': ['я хорошая и дружелюбная сорока. меня хвалят. ', 'бегает. прыгает. ', 'смешной и. расплывчатый', 'кто ты. кто я. '] 
})
toy_df

In [ ]:
lst_col = 'City' 

x = toy_df.assign(**{lst_col:toy_df[lst_col].str.split('.')})

my_dataframe = pd.DataFrame({col:np.repeat(x[col].values, x[lst_col].str.len()) for col in x.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

In [ ]:
my_dataframe

In [ ]:
my_dataframe['words'] = my_dataframe['City']
lst_col = 'words' 

x = my_dataframe.assign(**{lst_col:my_dataframe[lst_col].str.split(' ')})

my_final_dataframe = pd.DataFrame({col:np.repeat(x[col].values, x[lst_col].str.len()) for col in x.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

In [ ]:
my_final_dataframe

In [ ]:
my_final_dataframe['lemma'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].normal_form, axis = 1)
my_final_dataframe['form'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].tag, axis = 1)
my_final_dataframe['POS'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].tag.POS, axis = 1)

In [ ]:
my_final_dataframe

In [ ]:
my_final_dataframe.drop(my_final_dataframe[my_final_dataframe.words == ''].index, inplace=True)

In [ ]:
my_final_dataframe

## MAIN

NOUN	имя существительное

ADJF	имя прилагательное (полное)

ADJS	имя прилагательное (краткое)

COMP	компаратив

VERB	глагол (личная форма)

INFN	глагол (инфинитив)

PRTF	причастие (полное)

PRTS	причастие (краткое)

GRND	деепричастие

NUMR	числительное

ADVB	наречие

NPRO	местоимение-существительное

PRED	предикатив

PREP	предлог

CONJ	союз

PRCL	частица

INTJ	междометие

In [ ]:
def get_morphology(word):
    word1 = word.strip('"')
    df_word = my_final_dataframe.loc[my_final_dataframe['words'] == word1]
    list_cities = []
    for index, row in df_word.iterrows():
        answer_word = str('This word is in:'+ row['City'] + ', which comes from the article:' + row['EmployeeId'])
        list_cities.append(answer_word)
    return list_cities
    
def get_tagged(word, allowed_tags=['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']):
    compl_word = word.split('+')
    tag = set(allowed_tags)&set(compl_word)
    if not tag:
        return ['this POS tag is not in my vocabulary. Sorry!']
        
    else:
        current_word = morph.parse(compl_word[0])[0]
        current_word_lemma = current_word.normal_form
        current_word_POS = current_word.tag.POS
        df_word_POS = my_final_dataframe.loc[(my_final_dataframe['lemma'] == current_word_lemma) & (my_final_dataframe['POS'] == current_word_POS)]
        list_taggers = []
        for index, row in df_word_POS.iterrows():
            answer_tagger = str('This word is in:'+ row['City'] + ', which comes from the article:' + row['EmployeeId'])
            list_taggers.append(answer_tagger)
        return list_taggers        
    
def get_lemma(word):
    word_lem_parsed = morph.parse(word)[0]
    word_lem_lemma = word_lem_parsed.normal_form
    df_lemma = my_final_dataframe.loc[(my_final_dataframe['lemma'] == word_lem_lemma)]
    list_lemmas = []
    for index, row in df_lemma.iterrows():
        answer_lemma = str('This word is in:'+ row['City'] + ', which comes from the article:' + row['EmployeeId'])
        list_lemmas.append(answer_lemma)
    return list_lemmas    
    
def get_pos(word):
    df_pos = my_final_dataframe.loc[(my_final_dataframe['POS'] == word)]
    list_poses = []
    for index, row in df_pos.iterrows():
        answer_pos = str('This word is in:'+ row['City'] + ', which comes from the article:' + row['EmployeeId'])
        list_poses.append(answer_pos)
    return list_poses
    
def work_with_words(parsed_quiery, allowed_tags=['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']):
    #inputs list, outputs list
    for word in parsed_quiery:
        trick = word[0]
        if trick=='"':
            print('word form')
            for i in get_morphology(word):
                print(i)
        elif '+' in word:
            print('lemma+POS')
            for i in get_tagged(word):
                print(i)
        elif word in allowed_tags:
            print('pos')
            for i in get_pos(word):
                print(i)
        else:
            print('lemma')
            for i in get_lemma(word):
                print(i)
    
def work_with_quiery(quiery):
    #inputs input string, outputs dataframe
    quiery_parsed = quiery.split(' ')
    if len(quiery_parsed)>2 or len(quiery_parsed)==0:
        print('Oh, come on, we talked about it!')
    else:
        work_with_words(quiery_parsed, allowed_tags=['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ'])

In [ ]:
allowed_tags=['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']
def main():
    quiery = input('What will we search for? ')
    answer = work_with_quiery(quiery)
    answer

In [ ]:
main()